In [24]:
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
import pandas as pd

def load_google_sheet_to_dataframe_with_range(json_keyfile: str, spreadsheet_name: str, sheet_name: str, start_cell: str, column_row_index: int) -> pd.DataFrame:
    """
    Google 스프레드시트에서 특정 셀 주소를 기준으로 데이터를 Pandas DataFrame으로 로드하는 함수
    
    :param json_keyfile: JSON 서비스 계정 키 파일 경로
    :param spreadsheet_name: 스프레드시트 이름
    :param sheet_name: 워크시트 이름
    :param start_cell: 데이터가 시작하는 셀 주소 (예: "A3")
    :return: DataFrame
    """
    # Google Sheets 및 Drive API 스코프 설정
    SCOPES = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]
    
    # 인증 정보 생성
    creds = Credentials.from_service_account_file(json_keyfile, scopes=SCOPES)
    
    # Google Sheets 클라이언트 생성
    gc = gspread.authorize(creds)
    
    # 스프레드시트 열기
    spreadsheet = gc.open(spreadsheet_name)
    
    # 워크시트 열기
    worksheet = spreadsheet.worksheet(sheet_name)
    
    # A2:K2의 컬럼명 가져오기
    column_names = worksheet.row_values(column_row_index)
    
    # A3부터 데이터를 가져오기
    start_row = int(start_cell[1:])  # 행 번호
    data = worksheet.get_all_values()[start_row - 1:]  # 데이터만 가져오기

    # DataFrame으로 변환
    df = pd.DataFrame(data, columns=column_names)
    
    return df


# 입력 값 설정
json_keyfile = "../security/tennis-club-thirty-fourty-8b73bb1c5b50.json"
spreadsheet_name = "써티포티_건대_정기코트_마스터시트"  # 실제 스프레드시트 이름으로 변경
sheet_name = "matches"  # 실제 워크시트 이름으로 변경
start_cell = "A3"  # 데이터 시작 셀 위치
column_row_index = 2 # 칼럼명의 위치

# 데이터 로드
try:
    match_df = load_google_sheet_to_dataframe_with_range(json_keyfile, spreadsheet_name, sheet_name, start_cell, column_row_index)
    print("데이터 로드 성공!")
    print(match_df.head())  # DataFrame의 상위 5개 행 출력
    # 원하는 컬럼 리스트
    desired_columns = [
        'date', 'match_num', 'team1_forehand', 'team1_backhand', 
        'team2_forehand', 'team2_backhand', 'team1_score', 'team2_score', 
        'match_type', 'result', 'game_type'
    ]

    # 해당 컬럼들만 추출하여 새로운 DataFrame 생성
    match_df = match_df[desired_columns]
    
    players_df = load_google_sheet_to_dataframe_with_range(json_keyfile, spreadsheet_name, sheet_name='players', start_cell='B4', column_row_index=3)
    print("데이터 로드 성공!")
    print(players_df.head())  # DataFrame의 상위 5개 행 출력

    # 필요하면 엑셀로 저장
    # df.to_excel("output_with_columns.xlsx", index=False)
    # print("Excel 파일로 저장 완료: output_with_columns.xlsx")
except Exception as e:
    print(f"오류 발생: {e}")


데이터 로드 성공!
         date match_num team1_forehand team1_backhand team2_forehand  \
0  2024-09-08         1             성현             정현            예스욱   
1  2024-09-08         2            김연주             성현             유하   
2  2024-09-08         3             쥬드             태보            챨리심   
3  2024-09-08         4             유하             정현            김연주   
4  2024-09-08         5             성현             쥬드            챨리심   

  team2_backhand team1_score team2_score match_type result  ...              \
0            김연주           3           3         믹스   draw  ...               
1              쿤           1           4         혼복  team2  ...               
2             정현           2           4         남복  team2  ...               
3             태보           4           1         혼복  team1  ...               
4             태보           4           2         남복  team1  ...               

  알 수 없음 알 수 없음 알 수 없음 알 수 없음  
0      남      남      남      여  
1      여      남  

In [25]:
match_df

,date,match_num,team1_forehand,team1_backhand,team2_forehand,team2_backhand,team1_score,team2_score,match_type,result,game_type
0,2024-09-08,1,성현,정현,예스욱,김연주,3,3,믹스,draw,4게임
1,2024-09-08,2,김연주,성현,유하,쿤,1,4,혼복,team2,4게임
2,2024-09-08,3,쥬드,태보,챨리심,정현,2,4,남복,team2,4게임
3,2024-09-08,4,유하,정현,김연주,태보,4,1,혼복,team1,4게임
4,2024-09-08,5,성현,쥬드,챨리심,태보,4,2,남복,team1,4게임
...,...,...,...,...,...,...,...,...,...,...,...
249,,,,,,,,,,,
250,,,,,,,,,,,
251,,,,,,,,,,,
252,,,,,,,,,,,


In [20]:
import pandas as pd

# Initialize ELO ratings for all players based on gender
elo_ratings = {row['닉네임']: 1500 if row['성별'] == '남' else 1450 for _, row in players_df.iterrows()}

# Set K-factor for ELO calculations (commonly 32)
K = 32

def expected_score(team1_rating, team2_rating):
    """Calculate the expected score for team1 against team2."""
    return 1 / (1 + 10 ** ((team2_rating - team1_rating) / 400))

def update_elo(team1_rating, team2_rating, team1_score, team2_score):
    """Update the ELO ratings for both teams."""
    expected_team1 = expected_score(team1_rating, team2_rating)
    expected_team2 = 1 - expected_team1
    
    # Determine the actual score based on match result
    if team1_score > team2_score:
        actual_team1 = 1
        actual_team2 = 0
    elif team1_score < team2_score:
        actual_team1 = 0
        actual_team2 = 1
    else:
        actual_team1 = 0.5
        actual_team2 = 0.5
    
    # Update ratings
    new_team1_rating = team1_rating + K * (actual_team1 - expected_team1)
    new_team2_rating = team2_rating + K * (actual_team2 - expected_team2)
    
    return new_team1_rating, new_team2_rating

# Iterate over matches in date and match_num order
for index, row in match_df.sort_values(['date', 'match_num']).iterrows():
    # Get player ratings for team1 and team2 (average for doubles)
    team1_players = [row['team1_forehand'], row['team1_backhand']]
    team2_players = [row['team2_forehand'], row['team2_backhand']]
    
    team1_rating = (elo_ratings[team1_players[0]] + elo_ratings[team1_players[1]]) / 2
    team2_rating = (elo_ratings[team2_players[0]] + elo_ratings[team2_players[1]]) / 2
    
    # Update ratings based on the match result
    new_team1_rating, new_team2_rating = update_elo(
        team1_rating, team2_rating, row['team1_score'], row['team_2_score']
    )
    
    # Distribute the new team ratings back to individual players
    for player in team1_players:
        elo_ratings[player] += (new_team1_rating - team1_rating) / 2
    for player in team2_players:
        elo_ratings[player] += (new_team2_rating - team2_rating) / 2

# Add gender to the final ELO DataFrame
elo_df = pd.DataFrame([
    {'Player': player, 'ELO': rating, 'Gender': players_df.loc[players_df['닉네임'] == player, '성별'].values[0]}
    for player, rating in elo_ratings.items()
])


# display(elo_df.sort_values(by='ELO', ascending=False).reset_index(drop=True))

# Display ELO ratings grouped by gender and sorted in descending order within each group
elo_grouped_df = (
    elo_df.sort_values(by=['Gender', 'ELO'], ascending=[True, False])
    .groupby('Gender')
    .apply(lambda x: x.reset_index(drop=True))
)

display(elo_grouped_df)


KeyError: ''

In [8]:
import pandas as pd

# Assuming players_df and matches_df have already been loaded with your actual data
# For demonstration purposes, let's assume some sample data
# players_df = pd.read_excel('path_to_your_players_sheet.xlsx')
# matches_df = pd.read_excel('path_to_your_matches_sheet.xlsx')

# Get player nicknames from players DataFrame
players = players_df['닉네임'].values  # Get player nicknames from players DataFrame
matches = match_df  # Load matches DataFrame

# Initialize DataFrames for pairwise results
same_team_results = pd.DataFrame(columns=players, index=players).fillna('0전 0승 0패 0무')
opponent_results = pd.DataFrame(columns=players, index=players).fillna('0전 0승 0패 0무')

# Function to update results
def update_results(df, player1, player2, result):
    current = df.at[player1, player2]
    current_matches, current_wins, current_losses, current_draws = map(int, [
        current.split('전')[0],
        current.split(' ')[1].replace('승', ''),
        current.split(' ')[2].replace('패', ''),
        current.split(' ')[3].replace('무', '')
    ])
    
    current_matches += 1  # Increment the match count
    if result == 'win':
        current_wins += 1
    elif result == 'loss':
        current_losses += 1
    else:  # result == 'draw'
        current_draws += 1
    
    df.at[player1, player2] = f"{current_matches}전 {current_wins}승 {current_losses}패 {current_draws}무"

# Fill the DataFrames with results
for index, row in matches.iterrows():
    team1 = [row['team1_forehand'], row['team1_backhand']]
    team2 = [row['team2_forehand'], row['team2_backhand']]
    
    # Determine match result
    if row['team1_score'] > row['team_2_score']:
        result_team1 = 'win'
        result_team2 = 'loss'
    elif row['team1_score'] < row['team_2_score']:
        result_team1 = 'loss'
        result_team2 = 'win'
    else:
        result_team1 = 'draw'
        result_team2 = 'draw'

    # Update same team results
    for player1 in team1:
        for player2 in team1:
            if player1 != player2:
                update_results(same_team_results, player1, player2, result_team1)
                
    for player1 in team2:
        for player2 in team2:
            if player1 != player2:
                update_results(same_team_results, player1, player2, result_team2)

    # Update opponent results
    for player1 in team1:
        for player2 in team2:
            update_results(opponent_results, player1, player2, result_team1)
            update_results(opponent_results, player2, player1, result_team2)

# Clean the outputs to avoid multiple entries for the same match
same_team_results = same_team_results.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
opponent_results = opponent_results.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Set index and column names for clarity
same_team_results.index.name = 'Player'
same_team_results.columns.name = 'Teammate'

opponent_results.index.name = 'Player'
opponent_results.columns.name = 'Opponent'

# Display the results at the end
# print("Same Team Results:")
# display(same_team_results[:11])

# print("\nOpponent Results:")
# display(opponent_results[:11])

# 본인끼리 만나는 영역은 공백으로 설정
for df in [same_team_results, opponent_results]:
    for player in players:
        df.at[player, player] = ""

# 결과 출력
print("Same Team Results:")
display(same_team_results.iloc[:11,:11])

print("\nOpponent Results:")
display(opponent_results.iloc[:11,:11])

ModuleNotFoundError: No module named 'pandas'